In [2]:
import pandas as pd
import os
os.chdir("/Users/alandu/Documents/DFS/")
from datetime import datetime, timedelta
from pathlib import Path
import numpy as np

Create first/base case hist_fpts_mat_update matrix (only need to run once at beginning of season)

In [90]:
# first date
date = "2017-04-02"

results_mat = pd.read_csv("MLB/data_warehouse/" + date + "/player_results.csv")
results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']
results_mat = results_mat[['Name_Team', 'Actual Score']]
results_mat.rename(columns={'Actual Score':date}, inplace = True)
results_mat

results_mat.to_csv("MLB/data_warehouse/" + "2017-04-03" + "/hist_fpts_mat_update.csv", index=False, na_rep="NA")

Create hist_fpts_mat_update for rest of season

In [199]:
# rest of dates
date_list = [datetime.strptime("2017-04-04", '%Y-%m-%d') + timedelta(days=x) for x in range(0, 130)]

for date in date_list:
    file_path_date = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d'))
    if file_path_date.exists():
        date_tm1 = date - timedelta(days=1)
        
        # read in hist_fpts_mat_update from date t-1
        file_path_hist = Path("MLB/data_warehouse/" + date_tm1.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
        if file_path_hist.exists():
            hist_fpts_mat = pd.read_csv(str(file_path_hist))
        else:
            # fill in dates where folder is missing
            temp_date = date_tm1 - timedelta(days=1)
            file_path_hist = Path("MLB/data_warehouse/" + temp_date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
            num_missing = 1
            while (file_path_hist.exists() == False):
                temp_date = temp_date - timedelta(days=1)
                file_path_hist = Path("MLB/data_warehouse/" + temp_date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
                num_missing += 1
            hist_fpts_mat = pd.read_csv(str(file_path_hist))
            print(date)
            for i in range(0,num_missing):
                new_column = temp_date + timedelta(days=i+1)
                
                # add missing date
                new_column_m1 = new_column - timedelta(days=1)
                print(new_column_m1.strftime('%Y-%m-%d'))
                file_path_results = Path("MLB/data_warehouse/" + new_column_m1.strftime('%Y-%m-%d') + "/player_results.csv")
                if file_path_results.exists():
                    results_mat = pd.read_csv(str(file_path_results))
                    results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']
                    hist_fpts_mat = hist_fpts_mat.merge(results_mat[['Name_Team', 'Actual Score']], on="Name_Team", how='outer')
                    hist_fpts_mat.rename(columns={'Actual Score':new_column_m1.strftime('%Y-%m-%d')}, inplace = True)
                elif Path("MLB/data_warehouse/" + new_column_m1.strftime('%Y-%m-%d')).exists():
                    hist_fpts_mat[new_column_m1.strftime('%Y-%m-%d')] = float('NaN')
                
                # add previous dates NA
                hist_fpts_mat[new_column.strftime('%Y-%m-%d')] = float('NaN')
        
        # add fpts for date t
        file_path_results = Path("MLB/data_warehouse/" + date_tm1.strftime('%Y-%m-%d') + "/player_results.csv")
        if file_path_results.exists():
            results_mat = pd.read_csv(str(file_path_results))
            results_mat['Name_Team'] = results_mat['Player'] + "_" + results_mat['Team']

            new_hist_fpts_mat = hist_fpts_mat.merge(results_mat[['Name_Team', 'Actual Score']], on="Name_Team", how='outer')
            new_hist_fpts_mat.rename(columns={'Actual Score':date_tm1.strftime('%Y-%m-%d')}, inplace = True)
        else:
            new_hist_fpts_mat = hist_fpts_mat
            new_hist_fpts_mat[date_tm1.strftime('%Y-%m-%d')] = float('NaN')
        
        # write to file
        new_hist_fpts_mat.to_csv(str(file_path_date) + "/hist_fpts_mat_update.csv", index=False, na_rep="NA")

2017-07-10 00:00:00
2017-07-08
2017-07-14 00:00:00
2017-07-10
2017-07-11
2017-07-12
2017-07-17 00:00:00
2017-07-15
2017-08-09 00:00:00
2017-08-07


Create full covariance matrix for entire season

In [183]:
date_list = [datetime.strptime("2017-04-04", '%Y-%m-%d') + timedelta(days=x) for x in range(0, 130)]
for date in date_list:
    # create covariance matrix
    file_path_hist = Path("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/hist_fpts_mat_update.csv")
    if file_path_hist.exists():
        hist_fpts_mat_update = pd.read_csv(str(file_path_hist), header=None)
        hist_fpts_mat_update = hist_fpts_mat_update[1:] # remove first row (header)

        player_team_vec = hist_fpts_mat_update[0]
        player_team_vec = player_team_vec.reset_index(drop=True)

        hist_fpts_mat_update = hist_fpts_mat_update.drop(0, 1) # remove first column (names)
        hist_fpts_mat_update = hist_fpts_mat_update.astype(float)
        cov_mat_masked = np.ma.array(hist_fpts_mat_update, mask=np.isnan(hist_fpts_mat_update))
        cov_mat = np.ma.cov(cov_mat_masked, allow_masked=True)
        cov_mat = np.ma.getdata(cov_mat)
        
        # get list of team names in order
        column_teams = []
        for player_team in player_team_vec:
            column_teams.append(player_team.split('_')[1])

        # set covariance to 0 if not on same team
        for ind_row, player_team in enumerate(player_team_vec):
            row_team = player_team_vec[ind_row].split('_')[1]
            for ind_col, column_team in enumerate(column_teams):
                if row_team != column_team:
                    cov_mat[ind_row, ind_col] = 0.0
                    
        # TODO: set to 0 if less than played in less than 10% of games
                    
        # convert to pandas df to add column names
        cov_mat = pd.DataFrame(cov_mat)
        cov_mat.columns = player_team_vec
        
        # write to file
        cov_mat.to_csv("MLB/data_warehouse/" + date.strftime('%Y-%m-%d') + "/covariance_mat_update.csv", index=False, na_rep="NA")

Subset covariance matrix for each contest

In [80]:
file_path_hist = Path("MLB/data_warehouse/" + "2017-08-05" + "/hist_fpts_mat_update.csv")
file_path_date = Path("MLB/data_warehouse/" + "2017-08-05")
if file_path_hist.exists():
    # read in hist_fpts_mat_update
    hist_fpts_mat_update = pd.read_csv(str(file_path_hist), header=None)
    
    # read in hitters.csv
    hitters_df = pd.read_csv(str(file_path_date) + "/$40.00entry_MLB$250KSaturdaySlugfest[$50Kto1st]/hitters.csv")
    
    # find indicies of each player in hitters_df in hist_fpts_mat_update
    
    # read in covariance_mat_update
    covariance_mat_update = pd.read_csv(str(file_path_date) + "/covariance_mat_update.csv")
    

In [146]:
hitters_name_team = hitters_df['Name'] + "_" + hitters_df['teamAbbrev']
inds_match = []
for name_team in hitters_name_team:
    inds_match.append(hist_fpts_mat_update[hist_fpts_mat_update[0] == name_team].index.tolist())
inds_match = sum(inds_match, []) # unlist

In [151]:
temp = np.array(covariance_mat_update)
temp = temp[np.ix_(inds_match,inds_match)]
temp = pd.DataFrame(temp)
temp

,0,1,2,3,4,5,6,7,8,9,...,406,407,408,409,410,411,412,413,414,415
0,54.936364,-12.663524,0.000000,0.000000,0.000000,0.000000,3.396970,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,-12.663524,57.372025,0.000000,0.000000,0.000000,0.000000,7.761502,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,0.000000,0.000000,77.418416,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,0.000000,0.000000,0.000000,68.539604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,77.309889,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,-4.805830,0.000000,0.0,-9.496706,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,166.410336,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
6,3.396970,7.761502,0.000000,0.000000,0.000000,0.000000,48.785266,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.329670,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.379589,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.344044,...,0.000000,14.824812,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
